## Command imports

In [18]:
%pylab inline 
import os
import nibabel as nib
import numpy as np
import glob
from scipy.ndimage import label
from numpy.linalg import inv

Populating the interactive namespace from numpy and matplotlib


## Import mse IDs 

In [19]:
#subjects = np.genfromtxt("pbr_baselines.txt", dtype=str)
subjects = ["mse1328"]
print subjects[:5]

['mse1328']


## Set variables to lesion/segmentation files

In [20]:
for les_file in glob.glob("/data/henry7/PBR/subjects/mse1328/lesions_manual/*/alignment_lesions.nii.gz"):
    print les_file
for seg_file in glob.glob("/data/henry7/PBR/subjects/mse1328/masks/*/segmentation.nii.gz"):
    print seg_file

/data/henry7/PBR/subjects/mse1328/lesions_manual/ms56-mse1328-002-AX_T1_3D_IRSPGR/alignment_lesions.nii.gz
/data/henry7/PBR/subjects/mse1328/masks/ms56-mse1328-002-AX_T1_3D_IRSPGR/segmentation.nii.gz


## Obtain affines 

In [21]:
les_img = nib.load(les_file)
les_img.dataobj
seg_img = nib.load(seg_file)
seg_img.dataobj

les_data = les_img.get_data()
print les_data.shape
seg_data = seg_img.get_data()
print seg_data.shape

les_affine = les_img.get_affine()
print les_affine
seg_affine = seg_img.get_affine()
print seg_affine
inv_seg_affine = np.linalg.inv(seg_affine)
print inv_seg_affine

(256, 256, 150)
(256, 256, 256)
[[  -0.9375        0.            0.          123.6289978 ]
 [   0.            0.9375        0.          -93.36050415]
 [   0.            0.            1.          -44.42710114]
 [   0.            0.            0.            1.        ]]
[[  -1.            0.            0.          131.6289978 ]
 [   0.            0.            1.         -101.36050415]
 [   0.           -1.            0.          173.57290649]
 [   0.            0.            0.            1.        ]]
[[  -1.           -0.           -0.          131.6289978 ]
 [  -0.           -0.           -1.          173.57290649]
 [   0.            1.            0.          101.36050415]
 [   0.            0.            0.            1.        ]]


In [22]:
#lesion labels
les_labels, nles_labels = label(les_data==[1])

#segmentation labels
seg_brainstem_labels, n_seg_brainstem_labels = label(seg_data==[16])
seg_lcerebellumcortex_labels, n_seg_lcerebellumcortex_labels = label(seg_data==[8])
seg_rcerebellumcortex_labels, n_seg_rcerebellumcortex_labels = label(seg_data==[47])
seg_lcerebellumwm_labels, n_seg_lcerebellumwm_labels = label(seg_data==[7])
seg_rcerebellumwm_labels, n_seg_rcerebellumwm_labels = label(seg_data==[46])

In [23]:
nles_labels

11

In [24]:
n_seg_brainstem_labels

1

In [25]:
n = np.unique(les_labels)
print n

[ 0  1  2  3  4  5  6  7  8  9 10 11]


In [26]:
np.bincount(les_labels.ravel())

array([9829648,      69,     105,      54,      29,      69,      61,
            36,      47,     125,     111,      46])

## Function that gets coordinates from labels

In [35]:
def get_label_coord(labels,num_labels):
    all_label_coords = []
    if num_labels >= 2:
        for count in range(1, num_labels+1):
            cur_label_coords = []
            x,y,z = np.nonzero(labels==count)
            for count2 in range(len(x)):
                cur_coord = [x[count2],y[count2],z[count2]]
                cur_label_coords.append(cur_coord)
            all_label_coords.append(cur_label_coords)
    else:
        x,y,z = np.nonzero(labels==1)
        for count in range(len(x)):
            all_label_coords.append([x[count],y[count],z[count]])
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_les_coords[count]
    #    print ""
    return all_label_coords

## Function that converts coordinates to real space

In [36]:
def get_rs_coord(coordinates):
    all_rs_coords = []
    for count in range(len(coordinates)):
        cur_rs_coords = []
        for count2 in range(len(coordinates[count])):
            rs_coord = np.dot(les_affine, [coordinates[count][count2][0],
                                           coordinates[count][count2][1], 
                                           coordinates[count][count2][2],
                                           1])
            rs_coord_noone = [rs_coord[0],rs_coord[1],rs_coord[2]] #raw float
            #rs_coord_noone = [round(rs_coord[0],2),round(rs_coord[1],2),round(rs_coord[2],2)] #two decimal float
            #rs_coord_noone = [int(rs_coord[0]),int(rs_coord[1]),int(rs_coord[2])] #integers
            cur_rs_coords.append(rs_coord_noone)
        all_rs_coords.append(cur_rs_coords)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_rs_coords[count]
    #    print ""
    return all_rs_coords

## Function that converts coordinates to segmentation file coordinates

In [37]:
def get_seg_coord(coordinates):
    all_seg_coords = []
    for count in range(len(coordinates)):
        cur_seg_coords = []
        for count2 in range(len(coordinates[count])):
            seg_coord = np.dot(inv_seg_affine, [coordinates[count][count2][0],
                                                coordinates[count][count2][1], 
                                                coordinates[count][count2][2],
                                                1])
            #seg_coord_noone = [seg_coord[0],seg_coord[1],seg_coord[2]] #raw float
            seg_coord_noone = [round(seg_coord[0],2),round(seg_coord[1],2),round(seg_coord[2],2)] #two decimal float
            #seg_coord_noone = [int(seg_coord[0]),int(seg_coord[1]),int(seg_coord[2])] #integers
            cur_seg_coords.append(seg_coord_noone)
        all_seg_coords.append(cur_seg_coords)
    #for count in range(nlabels):
    #    print "lesion #", count+1, "", all_seg_coords[count]
    #    print ""
    return all_seg_coords

## Create midbrain coordinates

In [38]:
midbrain = []
midbrain.append(get_label_coord(seg_brainstem_labels,n_seg_brainstem_labels))
midbrain.append(get_label_coord(seg_lcerebellumcortex_labels,n_seg_lcerebellumcortex_labels))
midbrain.append(get_label_coord(seg_lcerebellumwm_labels,n_seg_lcerebellumwm_labels))
midbrain.append(get_label_coord(seg_rcerebellumcortex_labels,n_seg_rcerebellumcortex_labels))
midbrain.append(get_label_coord(seg_rcerebellumwm_labels,n_seg_rcerebellumwm_labels))

print len(midbrain[0])
    

18848
3


## Function that takes in coordinates for lesions, and outputs average; may not actually use this

In [39]:
def average_func(coordinates):
    averages_all = []
    for count in range(len(coordinates)):
        averages = []
        sumx=0;sumy=0;sumz=0
        for count2 in range(len(coordinates[count])):
            sumx += coordinates[count][count2][0]
            sumy += coordinates[count][count2][1]
            sumz += coordinates[count][count2][2]
        average_x = sumx / len(coordinates[count])
        average_y = sumy / len(coordinates[count])
        average_z = sumz / len(coordinates[count])
        averages = [average_x, average_y, average_z]
        averages_all.append(averages)
        #print count, averages
    return averages_all

In [40]:
average_les = average_func(get_label_coord(les_labels, nles_labels))
average_rs = average_func(get_rs_coord(get_label_coord(les_labels, nles_labels)))
average_seg = average_func(get_seg_coord(get_rs_coord(get_label_coord(les_labels, nles_labels))))

allcoord_les = get_label_coord(les_labels, nles_labels)
allcoord_rs = get_rs_coord(get_label_coord(les_labels, nles_labels))
allcoord_seg = get_seg_coord(get_rs_coord(get_label_coord(les_labels, nles_labels)))

print average_les[0]
print average_rs[0]
print average_seg[0]
print allcoord_les[0][0]
print allcoord_rs[0][0]
print allcoord_seg[0][0]


[78, 105, 89]
[50.191497802734375, 5.9601480235224189, 44.819275676340297]
[81.43898550724637, 128.7536231884058, 107.3221739130435]
[76, 105, 89]
[52.378997802734375, 5.076995849609375, 44.572898864746094]
[79.25, 129.0, 106.44]


### Not part of program, but this worked for one set of coordinates so loop data is compared to this

In [41]:
realspace_coordinates = np.dot(les_affine, [average_les[0][0],
                 average_les[0][1], 
                 average_les[0][2],
                 1])
average_les[0] 
realspace_coordinates
from numpy.linalg import inv
inv_seg_affine = np.linalg.inv(seg_affine)
freesurfer_matrix = np.dot(inv_seg_affine, [realspace_coordinates[0],
                                            realspace_coordinates[1], 
                                            realspace_coordinates[2],
                                            1])
print average_les[0]; print realspace_coordinates; print freesurfer_matrix

[78, 105, 89]
[ 50.5039978    5.07699585  44.57289886   1.        ]
[  81.125       129.00000763  106.4375        1.        ]


# Master file

for subject_count in range(len(subjects)):
    #show file directory
    for les_file in glob.glob("/data/henry7/PBR/subjects/%s/lesions_manual/*/alignment_lesions.nii.gz" % subjects[subject_count]):
        print les_file_1328
    for seg_file in glob.glob("/data/henry7/PBR/subjects/%s/masks/*/segmentation.nii.gz" % subjects[subject_count]):
        print seg_file_1328
    #loadfile
    les_img = nib.load(les_file)
    les_img.dataobj
    seg_img = nib.load(seg_file)
    seg_img.dataobj
    #print shape
    les_data = les_img.get_data()
    print les_data.shape
    seg_data = seg_img.get_data()
    print seg_data.shape
    #print affines
    les_affine = les_img.get_affine()
    print les_affine
    seg_affine = seg_img.get_affine()
    print seg_affine
    inv_seg_affine = np.linalg.inv(seg_affine)
    print inv_seg_affine
    #set labels
    #lesion labels
    les_labels, nles_labels = label(les_data==[1])
    #segmentation labels
    seg_les_labels, seg_nles_labels = label(seg_data==[4])
    #get coordinates from lesion
    def get_label_coord():
        all_label_coords = []
        for count in range(1, nles_labels+1):
            cur_label_coords = []
            x,y,z = np.nonzero(les_labels==count)
            for count2 in range(len(x)):
                cur_coord = [x[count2],y[count2],z[count2]]
                cur_label_coords.append(cur_coord)
            all_label_coords.append(cur_label_coords)
        #for count in range(nlabels):
        #    print "lesion #", count+1, "", all_label_coords[count]
        #    print ""
        return all_label_coords
    #get coordinates from lesion
    def get_rs_coord(coordinates):
        all_rs_coords = []
        for count in range(len(coordinates)):
            cur_rs_coords = []
            for count2 in range(len(coordinates[count])):
                rs_coord = np.dot(les_affine, [coordinates[count][count2][0],
                                                   coordinates[count][count2][1], 
                                                   coordinates[count][count2][2],
                                                   1])
                rs_coord_noone = [rs_coord[0],rs_coord[1],rs_coord[2]] #raw float
                #rs_coord_noone = [round(rs_coord[0],2),round(rs_coord[1],2),round(rs_coord[2],2)] #two decimal float
                #rs_coord_noone = [int(rs_coord[0]),int(rs_coord[1]),int(rs_coord[2])] #integers
                cur_rs_coords.append(rs_coord_noone)
            all_rs_coords.append(cur_rs_coords)
        #for count in range(nlabels):
        #    print "lesion #", count+1, "", all_rs_coords[count]
        #    print ""
        return all_rs_coords
    #get coordinates from real world
    def get_seg_coord(coordinates):
        all_seg_coords = []
        for count in range(len(coordinates)):
            cur_seg_coords = []
            for count2 in range(len(coordinates[count])):
                seg_coord = np.dot(inv_seg_affine, [coordinates[count][count2][0],
                                                        coordinates[count][count2][1], 
                                                        coordinates[count][count2][2],
                                                        1])
                #seg_coord_noone = [seg_coord[0],seg_coord[1],seg_coord[2]] #raw float
                seg_coord_noone = [round(seg_coord[0],2),round(seg_coord[1],2),round(seg_coord[2],2)] #two decimal float
                #seg_coord_noone = [int(seg_coord[0]),int(seg_coord[1]),int(seg_coord[2])] #integers
                cur_seg_coords.append(seg_coord_noone)
            all_seg_coords.append(cur_seg_coords)
        #for count in range(nlabels):
        #    print "lesion #", count+1, "", all_seg_coords[count]
        #    print ""
        return all_seg_coords